The goal of this notebook is to retrieve the times of the 88D and the kas that are the same so I can determine the proper distance between the two (lat/lon).

In [1]:
# import necessary stuff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import coords
import xarray as xr
import pyart
from datetime import timedelta
from radarcalc import *
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import cartopy
import gc

/var/folders/4f/9s_4__q16tx5t8vmww4mv03h0000gn/T/ipykernel_68270/3461312758.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# gps data for first ka
ka1gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt')
# scan data for first ka
ka1scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka1_scan_log.txt', dtype=str)

# dealiased vad data created from VADS.ipynb for ka 1, but not bearing corrected
vad_fileska1 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka1/*.nc'))

# sanity checks to make sure they're loading in:
#print(vad_fileska1)
#print(ka1gps)

In [56]:
ka1gps['hhmmss[UTC]']

0        183124
1        183125
2        183126
3        183127
4        183128
          ...  
31901     32353
31902     32354
31903     32355
31904     32356
31905     32357
Name: hhmmss[UTC], Length: 31906, dtype: int64

In [42]:
vad_fileska1

['/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka12205232.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205632.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205701.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205730.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205759.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205828.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205857.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205926.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205955.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523221718.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222551.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222620.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222649.nc',
 '/U

In [113]:
file = vad_fileska1[10]

In [114]:
file

'/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222551.nc'

In [115]:
# get the time of the file by grabbing the corresponding numbers from the file name
file[-9:-3]

'222551'

In [3]:
###### data for ka 2

ka2gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka2_20220523.txt')
# scan data for first ka
ka2scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka2_scan_log.txt', dtype=str)                    

# dealiased vad data created from VADS.ipynb for ka 2, but not bearing corrected
vad_fileska2 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka2/*.nc'))

# sanity checks to make sure they're loading in:
#print(vad_fileska2)
#print(ka2gps)

In [7]:
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track_KFDX.nc'
tobac_features_xr = xr.open_dataset(tobac_file)

In [8]:
tobac_features_xr

<xarray.Dataset>
Dimensions:                  (index: 653)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 5 ... 648 649 650 651 652
Data variables: (12/15)
    frame                    (index) int64 ...
    idx                      (index) int64 ...
    hdim_1                   (index) float64 ...
    hdim_2                   (index) float64 ...
    num                      (index) int64 ...
    threshold_value          (index) int64 ...
    ...                       ...
    projection_y_coordinate  (index) float64 ...
    projection_x_coordinate  (index) float64 ...
    latitude                 (index) float64 ...
    longitude                (index) float64 ...
    cell                     (index) int64 ...
    time_cell                (index) timedelta64[ns] ...

In [68]:
first_tobac_time = tobac_features_xr['timestr'].values[0]

In [109]:
# new = first_tobac_time[-8:]

In [110]:
# split = new.split(':')

In [111]:
# ''.join(split)

In [93]:
all_tobac_times = tobac_features_xr['timestr']

In [94]:
all_tobac_times

<xarray.DataArray 'timestr' (index: 653)>
array(['2022-05-23 20:52:31', '2022-05-23 20:52:31', '2022-05-23 21:02:09',
       ..., '2022-05-24 03:55:52', '2022-05-24 03:55:52',
       '2022-05-24 03:55:52'], dtype='<U19')
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 7 8 ... 645 646 647 648 649 650 651 652

Create a loop that will take the HH : MM : SS of each tobac time and put them in HHMMSS format.

In [105]:
times_list = []
for time in range(len(all_tobac_times)):
    file = all_tobac_times.values[time]
    time_select = file[-8:] # select the elements that represent the time
    time_split = time_select.split(':') # remove the : from the selected times
    time_join = ''.join(time_split) # join the time indeces with nothing separating them (aka, with '' joining them)
    times_list.append(time_join)

In [108]:
times_list[0:5]

['205231', '205231', '210209', '210209', '210730']

In [21]:
first_vad

<xarray.Dataset>
Dimensions:                    (time: 2039, range: 555, sweep: 2)
Coordinates:
  * time                       (time) datetime64[ns] 2022-05-23T20:57:34 ... ...
  * range                      (range) float32 0.0 18.0 ... 9.954e+03 9.972e+03
    azimuth                    (time) float32 ...
    elevation                  (time) float32 ...
Dimensions without coordinates: sweep
Data variables: (12/30)
    total_power                (time, range) float32 ...
    reflectivity               (time, range) float32 ...
    velocity                   (time, range) float32 ...
    spectrum_width             (time, range) float32 ...
    corrected_velocity         (time, range) float32 ...
    normalized_coherent_power  (time, range) float32 ...
    ...                         ...
    time_coverage_end          |S32 ...
    time_reference             |S32 ...
    volume_number              int32 ...
    platform_type              |S32 ...
    instrument_type            |S32 ...
    primary_axis               |S32 ...
Attributes: (12/15)
    Conventions:             CF/Radial instrument_parameters
    version:                 1.3
    title:                   
    institution:             
    references:              
    source:                  
    ...                      ...
    sigmet_task_name:        AF_VAD      
    sigmet_extended_header:  false
    time_ordered:            none
    rays_missing:            7
    field_names:             total_power, reflectivity, velocity, spectrum_wi...
    history:

In [24]:
first_vad['time']

<xarray.DataArray 'time' (time: 2039)>
array(['2022-05-23T20:57:34.000000000', '2022-05-23T20:57:35.000000000',
       '2022-05-23T20:57:35.000000000', ..., '2022-05-23T20:57:46.000000000',
       '2022-05-23T20:57:46.000000000', '2022-05-23T20:57:46.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time       (time) datetime64[ns] 2022-05-23T20:57:34 ... 2022-05-23T20:57:46
    azimuth    (time) float32 ...
    elevation  (time) float32 ...
Attributes:
    long_name:      time_in_seconds_since_volume_start
    standard_name:  time
    comment:        Coordinate variable for time. Time at the center of each ...

In [116]:
for the_file in vad_fileska1:
    if (the_file[-9:-3] in times_list) != True:   #makes sure the times are the same from both files
        continue